In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns

AttributeError: partially initialized module 'pandas' has no attribute '_pandas_parser_CAPI' (most likely due to a circular import)

In [ ]:
df = pd.read_csv("FACT.csv")
product_df = pd.read_csv("DIM_PRODUCT.csv", encoding='latin1')
customer_df = pd.read_csv("DIM_CUSTOMER.csv", encoding='latin1')
df = df.merge(product_df, on='Product ID', how="left")
df = df.merge(customer_df, on='Customer ID', how="left")

In [ ]:
df["Profit"] = df["Profit"].astype(str).str.replace(",", "").astype(float)

In [ ]:
df["Order Date"] = pd.to_datetime(df["Order Date"])
df["Month"] = df["Order Date"].dt.month
df["Day"] = df["Order Date"].dt.day

df["Sales*Quantity"] = df["Sales"] * df["Quantity"]

In [ ]:
df = df.drop_duplicates(subset=['Order ID'])

In [ ]:
categorical = ["Category","Segment"]
df = pd.get_dummies(df, columns=categorical, drop_first=True)

not_needed = ["Return_lable","Discount_Label","Order_to_Ship_Time","ORDERPRIORITY_ID",
              "MARKET_ID"]

features = ["Sales","Quantity","Discount","Month","Day","Sales*Quantity","MARKET_ID","ORDERPRIORITY_ID"] + [c for c in df.columns if "_" in c and c not in not_needed]

X = df[features]
y = df["Profit"]

['Sales',
 'Quantity',
 'Discount',
 'Month',
 'Day',
 'Sales*Quantity',
 'MARKET_ID',
 'ORDERPRIORITY_ID',
 'SHIPMODE_ID',
 'LOCATION_ID',
 'productkey_x',
 'customerkey_x',
 'productkey_y',
 'customerkey_y',
 'Category_Office Supplies',
 'Category_Technology',
 'Segment_Corporate',
 'Segment_Home Office']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
def normalize_features(X):
   X = (X - X.mean()) / X.std()
   return X

In [ ]:
X_train = normalize_features(X_train)
X_test = normalize_features(X_test)

In [ ]:
model_rf = RandomForestRegressor(
    n_estimators = 400,
    max_depth = 20,
    min_samples_leaf = 2,
    n_jobs = -1,
    random_state = 42
)
model_rf.fit(X_train, y_train)
y_pred_rf = model_rf.predict(X_test)

r2_rf = r2_score(y_test, y_pred_rf)
mse_rf = mean_squared_error(y_test, y_pred_rf)
print("Random Forest R2 Score:", round(r2_rf, 4))
print("Random Forest MSE:", round(mse_rf, 4))

Random Forest R2 Score: 0.7123
Random Forest MSE: 6671.0761


In [ ]:
model_xgb = xgb.XGBRegressor(n_estimators=200, max_depth=10, learning_rate=0.05, min_child_weight=2, n_jobs=-1, random_state=42)
model_xgb.fit(X_train, y_train)
y_pred_xgb = model_xgb.predict(X_test)

r2_xgb = r2_score(y_test, y_pred_xgb)
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
print("XGBoost R2 Score:", round(r2_xgb, 4))
print("XGBoost MSE:", round(mse_xgb, 4))

XGBoost R2 Score: 0.6387
XGBoost MSE: 8377.9773


In [ ]:
importances = pd.DataFrame({"Feature": features, "Importance": model_rf.feature_importances_})
print(importances.sort_values("Importance", ascending=False))

                     Feature  Importance
0                      Sales    0.494617
2                   Discount    0.270505
5             Sales*Quantity    0.069594
9                LOCATION_ID    0.025250
10              productkey_x    0.022112
13             customerkey_y    0.021716
4                        Day    0.020176
12              productkey_y    0.016013
11             customerkey_x    0.015019
3                      Month    0.014781
6                  MARKET_ID    0.008325
1                   Quantity    0.007333
8                SHIPMODE_ID    0.005852
7           ORDERPRIORITY_ID    0.004210
16         Segment_Corporate    0.001698
17       Segment_Home Office    0.001351
14  Category_Office Supplies    0.001076
15       Category_Technology    0.000373


Sales & Profit Relationship

In [ ]:
plt.figure(figsize=(8,6))
sns.scatterplot(data=df, x="Sales", y="Profit", alpha=0.6)
plt.title("Sales vs Profit", fontsize=14)
plt.xlabel("Sales")
plt.ylabel("Profit")
plt.show()

Profit Distribution

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df["Profit"], bins=40, kde=True, color="teal")
plt.title("Distribution of Profit", fontsize=14)
plt.xlabel("Profit")
plt.ylabel("Count")
plt.show()

Monthly Sales Trend

In [ ]:
plt.figure(figsize=(10,6))
monthly_sales = df.groupby("Month")["Sales"].sum()
sns.lineplot(x=monthly_sales.index, y=monthly_sales.values, marker="o")
plt.title("Monthly Sales Trend", fontsize=14)
plt.xlabel("Month")
plt.ylabel("Total Sales")
plt.grid(True, alpha=0.3)
plt.show()

Average Profit by Category

In [ ]:
plt.figure(figsize=(10,6))
category_profit = df.groupby("Category")["Profit"].mean().sort_values(ascending=False)
sns.barplot(x=category_profit.index, y=category_profit.values, palette="viridis")
plt.title("Average Profit by Product Category", fontsize=14)
plt.xlabel("Category")
plt.ylabel("Average Profit")
plt.show()